# Consultas Integradas

Este notebook tiene el fin de generar las consultas planteadas en el trabajo práctico integrador

#### Importo librerias

In [113]:
# Limpia todas las variables existentes
%reset -f

# Habilita autoreload para recargar automáticamente todos los módulos
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os
from pathlib import Path
import pandas as pd
from pymongo.errors import ConnectionFailure

from db_connections import client, db_neo4j, db_redis
from src import mongo, neo4j, utils, redis

ImportError: cannot import name 'contador' from 'src.mongo' (/work/src/mongo.py)

In [ ]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

✅ Conexión a MongoDB verificada.
✅ Conexión a Neo4j verificada.
✅ Conexión a Redis verificada.


## A.Mostrar los usuarios que visitaron “Bariloche”. 

In [ ]:
query = """ 
MATCH (U:Usuario)-[:VISITO]->(D:Destino)
WHERE D.ciudad='La Plata'
RETURN DISTINCT U.usuario_id AS id, U.nombre AS Nombre, U.apellido AS Apellido
"""

usuarios = neo4j.consulta(db_neo4j, query)
usuarios

,id,Nombre,Apellido
0,257,Abril,Pedrosa
1,373,Ciro,Toledo
2,312,Bibiana,Flores
3,60,Poncio,Escobar
4,129,Antonio,Madrid
5,221,Cleto,Ríos
6,301,Inés,Gabaldón
7,166,Alexandra,Oller
8,114,Eligia,Coronado
9,495,Gema,Rueda


### B.Mostrar los amigos de Juan que visitaron algún destino que visitó él, mostrar el nombre del Usuario y el destino. 

In [ ]:
import pandas as pd

# Pedimos el nombre del usuario
nombre_usuario = input("Introduce tu nombre para saber qué lugares visitaste con tus amigos: ")

# Query Cypher corregida: el parámetro se llama $nombre
query = """
MATCH (u:Usuario {nombre: $nombre})-[:AMIGO_DE]-(amigo:Usuario)
MATCH (u)-[:VISITO]->(d:Destino)<-[:VISITO]-(amigo)
RETURN 
    amigo.nombre AS Nombre,
    collect(DISTINCT d.ciudad) AS Destinos_Compartidos
ORDER BY Nombre
"""

# Mensaje de encabezado
print("\n" + "-"*60)
print(f"{nombre_usuario.upper()} TUS AMIGOS VISITARON ESTOS MISMOS DESTINOS QUE TÚ")
print("-"*60 + "\n")

# Ejecutamos la consulta usando la función consulta() que devuelve un DataFrame
usuarios = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

# Mostramos el resultado
if usuarios.empty:
    print("No se encontraron amigos que hayan visitado los mismos destinos.")
else:
    print(usuarios)



------------------------------------------------------------
JUAN TUS AMIGOS VISITARON ESTOS MISMOS DESTINOS QUE TÚ
------------------------------------------------------------

     Nombre                Destinos_Compartidos
0      Amor  [Córdoba, Mendoza, Azul, New York]
1      Blas  [Córdoba, Mendoza, Azul, New York]
2     Cleto  [Córdoba, Mendoza, Azul, New York]
3  Leopoldo  [Córdoba, Mendoza, Azul, New York]
4     Paula  [Córdoba, Mendoza, Azul, New York]
5  Wilfredo  [Córdoba, Mendoza, Azul, New York]
6     Óscar  [Córdoba, Mendoza, Azul, New York]


### C. Sugerir destinos a un usuario que no haya visitado él ni sus amigos. 

In [ ]:
nombre_usuario = input("Introduce tu nombre para saber qué lugares nuevos que no visitaste ni tu ni tus amigos: ")

query = """
MATCH (d:Destino)
WHERE 
NOT EXISTS {MATCH (d)<-[:VISITO]-(u:Usuario {nombre: $nombre})}
AND 
NOT EXISTS {MATCH (d)<-[:VISITO]-(:Usuario)-[:AMIGO_DE]-(:Usuario {nombre: $nombre})}
RETURN DISTINCT d.ciudad AS Destinos_No_Visitados
ORDER BY d.ciudad
"""

print("\n" + "-"*60)
print(f"DESTINOS NUEVOS PARA {nombre_usuario.upper()} Y SUS AMIGOS")
print("-"*60 + "\n")

destinos = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

if destinos.empty:
    print("No hay destinos nuevos disponibles.")
else:
    # Mostrar cada destino en lista
    for d in destinos['Destinos_No_Visitados']:
        print(f"- {d}")




------------------------------------------------------------
DESTINOS NUEVOS PARA JUAN Y SUS AMIGOS
------------------------------------------------------------

- Córdoba
- La Plata
- Mendoza
- Salta
- San Miguel de Tucumán


## d. Recomendar destinos basados en viajes de amigos.

In [ ]:
nombre_usuario = input("Introduce tu nombre para saber qué lugares te recomendamos: ")

query = """
MATCH (u:Usuario {nombre:$nombre})-[:AMIGO_DE]-(amigo:Usuario)
MATCH (amigo)-[:VISITO]->(d:Destino)
WHERE NOT (u)-[:VISITO]->(d)   
RETURN DISTINCT d.ciudad AS Destino_Recomendado
ORDER BY d.ciudad
"""

print("\n" + "-"*60)
print(f"RECOMENDACIONES PARA {nombre_usuario.upper()} EN FUNCIÓN DE SUS AMIGOS")
print("-"*60 + "\n")

destinos = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

if destinos.empty:
    print("No hay destinos recomendados nuevos para vos.")
else:
    for d in destinos['Destino_Recomendado']:
        print(f"- {d}")



------------------------------------------------------------
RECOMENDACIONES PARA JUAN EN FUNCIÓN DE SUS AMIGOS
------------------------------------------------------------

- Córdoba
- Mendoza
- Neuquén
- San Miguel de Tucumán


## e. Listar los hoteles en los destinos recomendados del punto anterior. 

In [ ]:
nombre_base = "clase"
coleccion = "hoteles"

# Lista de destinos recomendados
lista_destinos = destinos["Destino_Recomendado"].dropna().unique().tolist()

if not lista_destinos:
    print("No hay destinos recomendados disponibles.")
else:
    filtro = {"ciudad": {"$in": lista_destinos}}
    proyeccion = {"_id": 0, "nombre": 1, "ciudad": 1, "direccion": 1}

    # Obtener datos de Mongo y convertir a DataFrame
    cursor = mongo.obtener_cursor(
        nombre_base=nombre_base,
        nombre_coleccion=coleccion,
        filtro=filtro,
        proyeccion=proyeccion
    )

    hoteles = pd.DataFrame(list(cursor))

    if hoteles.empty:
        print("No se encontraron hoteles en los destinos recomendados.")
    else:
        # Ordenar por ciudad y luego por nombre de hotel
        hoteles.sort_values(by=["ciudad", "nombre"], inplace=True)
        print(hoteles.to_string(index=False))
       



                                          nombre                ciudad
            Aránzazu Carmona Belda S.L.N.E Hotel               Córdoba
     Banca Privada Madrid y asociados S.A. Hotel               Córdoba
                       Banco Barreda S.C.P Hotel               Córdoba
                        Despacho NLHV S.L. Hotel               Córdoba
Distribuciones Segovia & Asociados S.L.N.E Hotel               Córdoba
                      Finanzas EUJ S.L.N.E Hotel               Córdoba
             Francisca Aliaga Soto S.L.N.E Hotel               Córdoba
                      Grupo Castrillo S.A. Hotel               Córdoba
                           Grupo Cruz S.L. Hotel               Córdoba
                     Hermanos Arnau S.Com. Hotel               Córdoba
                   Hermanos Blasco S.L.N.E Hotel               Córdoba
                     Hermanos López S.A.T. Hotel               Córdoba
                         Hnos Arregui S.A. Hotel               Córdoba
      

## f.Ver las reservas en proceso, es decir que aún no están concretadas. 

## g. Listar los usuarios conectados actualmente. 

###### h. Mostrar los destinos con precio inferior a $100.000

## i. Mostrar todos los Hoteles de “Jujuy”.

In [ ]:
nombre_base = "clase"
coleccion = "hoteles"

# Lista de destinos recomendados
ciudad = ["Jujuy"]

if not lista_destinos:
    print("No hay destinos recomendados disponibles.")
else:
    filtro = {"ciudad": {"$in": ciudad}}
    proyeccion = {"_id": 0, "nombre": 1, "ciudad": 1, "direccion": 1}

    # Obtener datos de Mongo y convertir a DataFrame
    cursor = mongo.obtener_cursor(
        nombre_base=nombre_base,
        nombre_coleccion=coleccion,
        filtro=filtro,
        proyeccion=proyeccion
    )

    hoteles = pd.DataFrame(list(cursor))

    if hoteles.empty:
        print("No se encontraron hoteles en los destinos recomendados.")
    else:
        # Ordenar por ciudad y luego por nombre de hotel
        hoteles.sort_values(by=["nombre"], inplace=True)
        print(hoteles.to_string(index=False))
       

No se encontraron hoteles en los destinos recomendados.


###### j. Mostrar la cantidad de hoteles de un destino que guste. 

In [ ]:
nombre_base = "clase"
coleccion = "hoteles"
agrupacion="destino"
campo_calculo="nombre"
operacion="sum"
cursor=contador(nombre_base, coleccion, agrupacion, campo_calculo, operacion)
df=pd.DataFrame(list(cursor))
df

AttributeError: module 'src.mongo' has no attribute 'contador'

###### k. Mostrar las actividades de “Ushuaia” del tipo “aventura”.

###### l. Mostrar la cantidad de reservas concretadas de cada usuario. Mostrar el usuario y la cantidad 

#### Estadísticas

###### i.Destino más visitado

###### ii.Hotel más barato

###### iii.Actividad más popular.

#### Modificaciones

###### a.Incrementar el precio de las actividades de Tucuman en 5% 

###### b. Agregar al hotel id=1 el servicio de SPA 

###### c. Eliminar el destino que desee

###### d. Eliminar un usuario que desee 

###### e. Eliminar las relaciones AMIGO_DE para un usuario que quiera. 